# Diagonal

https://github.com/numpy/numpy/issues/4965

In [17]:
import numpy as np
i,j,k = 10, 20, 30
a = np.random.rand(i, j, k)
b = np.random.rand(j,k)
out = np.zeros((i, j, i, j, i))
from numpy.lib.stride_tricks import as_strided
out_view = as_strided(out, shape=(i, j),
                       strides=(out.strides[0]+out.strides[2]+out.strides[4],
                                out.strides[1]+out.strides[3]))
np.einsum('ijk,jk->ij', a, b, out=out_view)

array([[6.67763676, 8.6725301 , 9.43029008, 6.82085536, 6.49409598,
        7.91976606, 7.96066167, 4.94796388, 6.24192742, 6.18046678,
        7.82960024, 6.34933661, 8.54120025, 6.66141176, 6.79120224,
        8.42350336, 8.09780888, 7.13564588, 5.55493242, 7.38973143],
       [8.33927416, 6.91245164, 7.77343637, 6.66466219, 7.15076897,
        7.13045607, 8.17431163, 5.2105344 , 6.16003568, 8.55064905,
        7.12378983, 5.9794665 , 7.99463899, 5.93752891, 5.9585692 ,
        7.66971191, 5.78393379, 8.18919643, 7.5164932 , 9.22159113],
       [8.62119299, 8.5699627 , 6.66215826, 6.75950928, 7.00660704,
        7.45416771, 9.96813931, 6.36654815, 7.59810672, 9.0045912 ,
        6.78517769, 5.5306725 , 7.66792429, 6.09231795, 7.20945146,
        8.03068937, 8.02398489, 6.86382357, 7.24752575, 6.91764794],
       [7.04443289, 8.84058813, 6.65823554, 8.45563694, 8.38267821,
        7.96527793, 8.73538472, 4.57095325, 8.5418031 , 7.97512235,
        5.51989644, 5.34109195, 8.37074068, 8

In [25]:
i = 3
x = np.arange(i)
out = np.zeros((i,i,i,i))
out_view = as_strided(out, shape=(i,),
                     strides=((sum(out.strides),)))
out_view[:] = x

In [26]:
out

array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]],


       [[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 1., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]],


       [[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 2.]]]])

In [27]:
np.einsum("ijij->ij", out)

array([[0., 0., 0.],
       [0., 1., 0.],
       [0., 0., 2.]])